### Import of some Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import cvxopt
import warnings
warnings.filterwarnings("ignore" )

### Import of DataSet

In [2]:
#Import of DataSet
X = np.loadtxt('Xtr.csv', skiprows=1, usecols=(1,), dtype=str, delimiter=',')
y = np.loadtxt('Ytr.csv', skiprows=1, usecols=(1,), dtype=int, delimiter=',')
Xtest = np.loadtxt('Xte.csv', skiprows=1, usecols=(1,), dtype=str, delimiter=',')

X_tr_vector = np.loadtxt('Xtr_vectors.csv', skiprows=1, usecols=(), dtype=str, delimiter=',')
X_test_vector = np.loadtxt('Xte_vectors.csv', skiprows=1, usecols=(), dtype=str, delimiter=',')

X_full = np.hstack([X, Xtest])
y = 2*y - 1.

In [3]:
# X.shape,y.shape, X_test.shape

### Break up a sequence into subsequences

In [4]:
def get_kmers(sequence, kmer_size=4):
    return [sequence[i: i+kmer_size] for i in range(len(sequence) - kmer_size + 1)]

def base2int(c):
    return {'A': 0, 'C': 1, 'G': 2, 'T': 3}.get(c, 0)

def index(kmer):
    # Transform the kmer into sequence of character indices
    return [base2int(c) for c in kmer]

In [5]:
def spectral_embedding(sequence):
    kmers = get_kmers(sequence)
    multiplier = 4 ** np.arange(len(kmer))[::-1]
    kmer_indices = [multiplier.dot(index(kmer)) for kmer in kmers]
    one_hot_vector = np.zeros(4**kmer_size).astype(int)
    for kmer_index in kmer_indices:
        one_hot_vector[kmer_index] += 1
    return one_hot_vector

In [6]:
kmer_size = 4
kmer = X[0][0:kmer_size] 
base_indices = np.array([base2int(base) for base in kmer])
# base_indices
multiplier = 4 ** np.arange(len(kmer))
# multiplier
kmer_index = multiplier[::-1].dot(base_indices)
# kmer_index

### Enconding of X_train

In [7]:
liste2=[]
for i in X:
    sequence = i
    liste2.append(spectral_embedding(sequence))

In [8]:
l=np.array(liste2)
X_train=l/(len(sequence)-kmer_size+1)

In [9]:
X_train

array([[0.00668896, 0.00334448, 0.01003344, ..., 0.        , 0.01003344,
        0.01003344],
       [0.00334448, 0.        , 0.01337793, ..., 0.00334448, 0.00668896,
        0.00668896],
       [0.00668896, 0.00334448, 0.00668896, ..., 0.00334448, 0.00668896,
        0.01003344],
       ...,
       [0.00334448, 0.00334448, 0.00334448, ..., 0.01003344, 0.01672241,
        0.0367893 ],
       [0.02341137, 0.01003344, 0.01003344, ..., 0.00334448, 0.        ,
        0.00334448],
       [0.        , 0.00334448, 0.        , ..., 0.00668896, 0.01672241,
        0.00668896]])

### Enconding of X_test

In [10]:
kmer2 = Xtest[0][0:kmer_size] 
base_indices2 = np.array([base2int(base) for base in kmer2])
# base_indices
multiplier2 = 4 ** np.arange(len(kmer2))
# multiplier
kmer_index2 = multiplier2[::-1].dot(base_indices2)

In [11]:
def spectral_embedding_test(sequence2):
    kmers = get_kmers(sequence2)
    multiplier = 4 ** np.arange(len(kmer2))[::-1]
    kmer_indices = [multiplier.dot(index(kmer2)) for kmer2 in kmers]
    one_hot_vector = np.zeros(4**kmer_size).astype(int)
    for kmer_index in kmer_indices:
        one_hot_vector[kmer_index] += 1
    return one_hot_vector

In [12]:
liste_=[]
for j in Xtest:
    sequence2 = j
    liste_.append(spectral_embedding_test(sequence2))

In [13]:
d=np.array(liste_)
X_test=d/(len(sequence2)-kmer_size+1)

In [14]:
X_test

array([[0.        , 0.        , 0.        , ..., 0.00334448, 0.00334448,
        0.00668896],
       [0.01003344, 0.        , 0.01337793, ..., 0.00668896, 0.01003344,
        0.00668896],
       [0.        , 0.00668896, 0.00334448, ..., 0.00668896, 0.00668896,
        0.01337793],
       ...,
       [0.        , 0.00334448, 0.01003344, ..., 0.        , 0.00334448,
        0.        ],
       [0.00334448, 0.00334448, 0.00334448, ..., 0.00334448, 0.        ,
        0.00334448],
       [0.00668896, 0.01003344, 0.        , ..., 0.01003344, 0.        ,
        0.        ]])

In [15]:
#Split y_train and y_test
#you can shuffle y 

y_train,y_test=y,y[1000:]

In [16]:
#All values are converted to float

X_train=X_train.astype(float) 
y_train=y_train.astype(float) 
X_test=X_test.astype(float) 

In [17]:
# Utilities
def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    #cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
    return np.array(solution['x']).flatten()

solve_qp = cvxopt_qp

In [18]:
# Prediction error
def error(ypred, ytrue):
    e = (ypred != ytrue).mean()
    return e

def add_column_ones(X):
    n = X.shape[0]
    return np.hstack([X, np.ones((n, 1))])

# Visualization
# References: https://scikit-learn.org/stable/auto_examples/svm/plot_iris.html
def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(classifier, xx, yy, add_intercept=False, **params):
    """Plot the decision boundaries for a classifier.
    
    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    X = np.c_[xx.ravel(), yy.ravel()]
    Z = classifier.predict(X)#, fit_intercept=add_intercept)
    Z = Z.reshape(xx.shape)
    out = plt.contourf(xx, yy, Z, **params)
    return out

def plot_decision_function(classifier, X_train, y_train, title='', add_intercept=False):
    fig = plt.figure(figsize=(8,7))
    X0, X1 = X_train[:, 0], X_train[:, 1]
    xx, yy = make_meshgrid(X0, X1)
    plot_contours(classifier, xx, yy, cmap=plt.cm.GnBu, alpha=0.5, add_intercept=add_intercept)
    plt.scatter(X0, X1, c=y_train, cmap=plt.cm.GnBu, s=20, edgecolors='k')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.title(title)
    plt.show()

### kernel implementations

In [19]:

def rbf_kernel(X1, X2, sigma=10):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the RBF kernel with parameter sigma
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    sigma: float
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis = -1)
    X1_norm = np.sum(X1 ** 2, axis = -1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

def linear_kernel(X1, X2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the linear kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return X1.dot(X2.T)

def polynomial_kernel(X1, X2, degree=3):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the polynomial kernel of degree `degree`
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return (X1.dot(X2.T) + 1)**degree

### Kernel Methods

In [20]:

class KernelMethodBase(object):
    '''
    Base class for kernel methods models
    
    Methods
    ----
    fit
    predict
    fit_K
    predict_K
    '''
    kernels_ = {
        'linear': linear_kernel,
        'polynomial': polynomial_kernel,
        'rbf': rbf_kernel,
        # 'mismatch': mismatch_kernel,
    }
    def __init__(self, kernel='linear', **kwargs):
        self.kernel_name = kernel
        self.kernel_function_ = self.kernels_[kernel]
        self.kernel_parameters = self.get_kernel_parameters(**kwargs)
        self.fit_intercept_ = False
        
    def get_kernel_parameters(self, **kwargs):
        params = {}
        if self.kernel_name == 'rbf':
            params['sigma'] = kwargs.get('sigma', 1.)
        if self.kernel_name == 'polynomial':
            params['degree'] = kwargs.get('degree', 3)
        return params

    def fit_K(self, K, y, **kwargs):
        pass
        
    def decision_function_K(self, K):
        pass
    
    def fit(self, X, y, fit_intercept=False, **kwargs):

        if fit_intercept:
            X = add_column_ones(X)
            self.fit_intercept_ = True
        self.X_train = X
        self.y_train = y

        K = self.kernel_function_(self.X_train, self.X_train, **self.kernel_parameters)

        return self.fit_K(K, y, **kwargs)
    
    def decision_function(self, X):

        if self.fit_intercept_:
            X = add_column_ones(X)

        K_x = self.kernel_function_(X, self.X_train, **self.kernel_parameters)

        return self.decision_function_K(K_x)

    def predict(self, X):
        pass
    
    def predict_K(self, K):
        pass

### Kernel SVM

In [21]:

#Soft-margin dual problem

def svm_dual_soft_to_qp_kernel(K, y, C=1):
    n = K.shape[0]
    assert (len(y) == n)
        
    # Dual formulation, soft margin
#     P = np.diag(y).dot(K).dot(np.diag(y))
    P=K*y*y[:,None]
    # As a regularization, we add epsilon * identity to P
    eps = 1e-12
    P += eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y[np.newaxis, :]
    b = np.array([0.])
    return P, q, G, h, A, b

K = linear_kernel(X_train, X_train)
alphas = solve_qp(*svm_dual_soft_to_qp_kernel(K, y_train, C=1.))

In [22]:
class KernelSVM(KernelMethodBase):
    '''
    Kernel SVM Classification
    
    Methods
    ----
    fit
    predict
    '''
    def __init__(self, C=0.1, **kwargs):
        self.C = C
        super().__init__(**kwargs)
    
    def fit_K(self, K, y, tol=1e-3):
        # Solve dual problem
        self.alpha = solve_qp(*svm_dual_soft_to_qp_kernel(K, y, C=self.C))
        
        # Compute support vectors and bias b
        sv = np.logical_and((self.alpha > tol), (self.C - self.alpha > tol))
        self.bias = y[sv] - K[sv].dot(self.alpha * y)
        self.bias = self.bias.mean()

        self.support_vector_indices = np.nonzero(sv)[0]
        
        return self
        
    def decision_function_K(self, K_x):
        return K_x.dot(self.alpha * self.y_train) + self.bias

    def predict(self, X):
        return np.sign(self.decision_function(X))

### Test our Kernel SVM

In [23]:

kernel = 'rbf'
sigma = 1.
degree = 3
C =200
tol = 1e-1
model = KernelSVM(C=C, kernel=kernel, sigma=sigma, degree=degree)
y_pred = model.fit(X_train, y_train, tol=tol).predict(X_test)
# plot_decision_function(model, X_test, y_test,title='SVM {} Kernel'.format(kernel))
print('Test error: {:.2%}'.format(error(y_pred, y_test)))

Test error: 48.50%


In [24]:
y_pred

array([-1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1., -1.,
        1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,
       -1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,
       -1.,  1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,
       -1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,
        1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,
        1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,
        1., -1., -1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,
       -1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.,  1.,
        1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,
        1., -1., -1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1., -1., -1.,
        1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.,
        1.,  1., -1.,  1.,  1.,  1., -1., -1., -1., -1.,  1., -1., -1.,
        1., -1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,  1

In [ ]:
pred = np.where(y_pred == -1, 0, 1) #To convert all -1 values to 0
Id=(np.arange(1000)+1).reshape(-1,1) #Creation of indexes
y_save = np.c_[Id,pred] #Concatenate Indexes and y_pred
y_save[:10]

In [ ]:
# Save as a csv file
np.savetxt('sample_prediction9.csv', y_save,
           delimiter=',', header='Id,Covid', fmt='%i', comments='')